### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
# !pip install langchain

In [3]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPEN_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
# groq_api_key

In [4]:
# !pip install langchain_groq

In [7]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
# model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model=ChatGroq(model="openai/gpt-oss-20b",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027B4443DF00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027B4443E1D0>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
# !pip install langchain_core

In [11]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to Nepali"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [12]:
result

AIMessage(content='नमस्ते, तपाईंलाई कस्तो छ?', additional_kwargs={'reasoning_content': 'The user wants translation of "Hello How are you?" from English to Nepali. So produce Nepali translation. Should be "नमस्ते, तपाईंलाई कस्तो छ?" or "नमस्कार, तपाईंलाई कस्तो छ?" The user didn\'t specify tone. Use standard. Provide translation.'}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 87, 'total_tokens': 169, 'completion_time': 0.082565133, 'prompt_time': 0.015741526, 'queue_time': 0.181652822, 'total_time': 0.098306659, 'completion_tokens_details': {'reasoning_tokens': 62}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--e3acf8fa-fe80-4b4d-95e8-266876be79b9-0', usage_metadata={'input_tokens': 87, 'output_tokens': 82, 'total_tokens': 169, 'output_token_details': {'reasoning': 62}})

In [13]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'नमस्ते, तपाईंलाई कस्तो छ?'

In [14]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'नमस्ते, तपाईंलाई कस्तो छ?'

In [15]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [16]:
result=prompt.invoke({"language":"Nepali ","text":"Hello"})

In [17]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into Nepali :', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [18]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Nepali","text":"Hello"})

'नमस्ते'

In [19]:
# !pip install streamlit